<a href="https://colab.research.google.com/github/SangHoon-Chae/ProblemSolving/blob/main/pbFileMaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import dataDivide as dD
#import dataExtractFunc as dataExt
import dataExtractFunc2 as dataExt2
import dataExtractFunc_AG_aug_userID as dataExt3
import sys, os
import os.path
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
sys.path.append(os.pardir)
from keras.optimizers import SGD
# import dataExtractFunc as dataExt
import matplotlib.pyplot as plt

np.random.seed(1337)  # for reproducibility

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

batch_size = 128
num_classes = 5
epochs = 30
romIntegral = 0

#img_cols means the size of time-window
img_rows, img_cols = 6, 90
time_window = 30

def shuffle_dataset(x, t):
    """
    데이터셋을 뒤섞는다.

    Parameters
    ----------
    x : 훈련 데이터
    t : 정답 레이블

    Returns
    -------
    x, t : 뒤섞은 훈련 데이터와 정답 레이블
    """
    permutation = np.random.permutation(x.shape[0])
    x = x[permutation,:] if x.ndim == 2 else x[permutation,:,:,:]
    t = t[permutation]

    return x, t



def extract_accelerometer_data(file_path):
    ax, ay, az = [], [], []  # 각각의 데이터를 저장할 배열 초기화


    with open(file_path, 'r') as file:
        lines = file.readlines()

        # 특정 키워드 이후 데이터 읽기 시작
        start_reading = False
        for line in lines:
            line = line.strip('')  # 줄 끝 공백 제거
#            print("Processing line: %s" %line)  # 디버그: 현재 처리 중인 줄

            if line == "End of file":
                print("Reached end of file.")
                break  # 파일 끝이면 종료

            if start_reading:
                # 데이터를 공백 기준으로 나눠서 ax, ay, az로 분리
                try:
                    x, y, z = map(float, line.split("\t"))
                    ax.append(x)
                    ay.append(y)
                    az.append(z)
                except ValueError:
                    print("잘못된 데이터 형식: %s" % line)
#                    print(f"잘못된 데이터 형식: {line}")

            if line == "Accelerometer\n":
                print("Found 'accelerometer' keyword.")
                start_reading = True  # accelerometer 이후부터 읽기 시작

    return ax, ay, az


# 파일 경로 설정
file_path = 'sensorData2024-11-27-06-16-07.txt'

# 함수 호출 및 결과 저장
ax, ay, az = extract_accelerometer_data(file_path)

gx, gy, gz = [], [], []
gx = ax
gy = ay
gz= az

ax = np.asarray(ax);ay = np.asarray(ay);az = np.asarray(az);
gx = np.asarray(gx);gy = np.asarray(gy);gz = np.asarray(gz);

ax = ax.reshape(-1,len(ax));ay = ay.reshape(-1,len(ay));az = az.reshape(-1, len(az));
gx = gx.reshape(-1,len(gx));gy = gy.reshape(-1,len(gy));gz = gz.reshape(-1, len(gz));

data = np.concatenate((ax,ay,az, gx, gy, gz), axis = 0);

# data = data[:, 0:data.shape[1]-(data.shape[1])%img_cols]
# data = np.hsplit(data, int(data.shape[1]/img_cols))
# data = np.asarray(data)

x_noExer, t_noExer = dD.dataDivideByTimdWindow(data, 30, img_cols, 0, 5);
x_flex, t_flex = dD.dataDivideByTimdWindow(data, 30, img_cols, 1, 5);
x_wall, t_wall = dD.dataDivideByTimdWindow(data, 30, img_cols, 2, 5);
x_actSca, t_actSca = dD.dataDivideByTimdWindow(data, 30, img_cols, 3, 5);
x_towel, t_towel = dD.dataDivideByTimdWindow(data, 30, img_cols, 4, 5);

x_train_result = np.concatenate((x_noExer, x_flex, x_wall, x_actSca,  x_towel))
t_train_result = np.concatenate((t_noExer, t_flex, t_wall, t_actSca, t_towel))
x_train_result, t_train_result = shuffle_dataset(x_train_result, t_train_result)

x_test_result = x_train_result[0:30]
t_test_result = t_train_result[0:30]

x_train_result = x_train_result[31:len(x_train_result)]
t_train_result = t_train_result[31:len(t_train_result)]

if K.image_data_format() == 'channels_first':
    x_train_result = x_train_result.reshape(x_train_result.shape[0], 1, img_rows, img_cols)
    x_test_result = x_test_result.reshape(x_test_result.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train_result = x_train_result.reshape(x_train_result.shape[0], img_rows, img_cols, 1)
    x_test_result = x_test_result.reshape(x_test_result.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

print('x_train shape:', x_train_result.shape)
print(x_train_result.shape[0], 'train samples')
print(x_test_result.shape[0], 'test samples')

#t_train_result = keras.utils.to_categorical(t_train_result, num_classes)
#t_test_result = keras.utils.to_categorical(t_test_result, num_classes)
#%%
# set learning phase to 0
model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (2, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(1, 1)))
#model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

history = model.fit(x_train_result, t_train_result,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test_result, t_test_result))

# list all data in history
print(history.history.keys())

# # summarize history for accuracy
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

# # summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

score = model.evaluate(x_test_result, t_test_result, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

MODEL_NAME = "keras_test"

tf.saved_model.save(model, 'out/' + MODEL_NAME)

del model  # deletes the existing model


Found 'accelerometer' keyword.
잘못된 데이터 형식: End of file

x_train shape: (64, 6, 90, 1)
64 train samples
30 test samples
Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.2031 - loss: 1.8373 - val_accuracy: 0.2000 - val_loss: 1.9755
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step - accuracy: 0.2031 - loss: 1.8202 - val_accuracy: 0.2000 - val_loss: 1.9601
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 295ms/step - accuracy: 0.2031 - loss: 1.8041 - val_accuracy: 0.2000 - val_loss: 1.9453
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step - accuracy: 0.2031 - loss: 1.7886 - val_accuracy: 0.2000 - val_loss: 1.9309
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 331ms/step - accuracy: 0.2031 - loss: 1.7736 - val_accuracy: 0.2000 - val_loss: 1.9171
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step - accuracy: 0.2031 - loss: 1.7595 - val_accuracy: 0.2000 - val_loss: 1.9039
Epoch 7/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step - accuracy: 0.2031 - loss: 1.7461 - val_accuracy: 0.2000 - val_loss: 1.8916
Epoch 8/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step - accuracy: 0.2031 - loss: 1.7336 - val_accuracy: 0.2000 - val_loss: 1.8798
Epoch 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [16]:
print(x_train_result.shape[0])

64
